# MSIII Computations

We just changed several things in our MSII orbit to address issues from the second milestone and make instructor-recommended corrections. A brief summary is below, but the MSII_Base_Orbit_Determination_corrected.ipynb and MSII_corrected.ipynb files can be consulted for more specific details about the implementation.
- Used maximum latitude instead of average (see Formula_Sheet.py)
- Changed max resolution to 3.99m (see MSIII_useful_consts.py)
- Added visits-per-day metric (see MSIII_Constellation.py)
- Used visits-per-day as metric for orbit template (MSII_Base_Orbit_Determination_corrected.ipynb)
    - This changed period to 2h and altered COEs significantly
- Forced the cheapest camera for budget reasons (see MSIII_Components.py)
    - This allowed cheaper structure and possibly changed ADCS
- Recomputed constraints to ensure they are met (se MSII_corrected.ipynb)

Note that the comments in all of these files have not necessarily changed, but the functionality has. The emphasis was on quick corrections to move on to MSIII, but the changes are fairly minor and should be understandeable without updated comments.

Now, we have several orders of business before we can truly begin MSIII.

In [16]:
#  import everything (our entire library)
from MSIII_Components import *
from MSIII_useful_consts import *
from MSIII_Constellation import MSIII_Constellation, MSIII_Satellite
from MSIII_Constraints import *
from MSIII_Orbit import MSIII_Orbit
from pretty_outputs import indent_section
from Formula_Sheet import *
from CoolGeometry import *

### Several orders of business...
We need to load the MSII corrected constellation into our program. Then, we need to somewhat nerf (make less powerful) it in order to meet the constraint of having all the same COEs except for true anomaly. Be advised that this will have HORRENDOUS impacts on performance. Now, the points only cross an orbital plane at most 2 times per day. Before, they crossed one at most six times per day. Having a single orbital plane for the entire constellation is the single worst thing you could do for point coverage. 

But, we must see to the needs of the Space Force. Single orbital plane it is...

In [17]:
# Here is the summary of our starting corrected MSII constellation
"""
Found constellation with 0.04225% coverage
Constellation #1:
	Payload:   Tiny Cam
	Structure: Option I
	ADCS:      Option II
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 300.375
		w    : 90
		v    : 126.19563368939261
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 310.375
		w    : 90
		v    : 132.1759947994688

Constellation #1 Comprehensive Summary: 
	Constellation #1 Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation #1 Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
		ORBIT 2
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
		ORBIT 3
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      2760.7213975942896 km
			Min Altitude:      601.0 km
			Is Geosynchronous: False
		
	
	Constellation #1 Budgeting:
		Margin Meets Budget:  True
		Certain Cost:        $16365000
		Est. Cost:           $20540000.000000004
		Cost With Margin:    $22594000.000000007
		Budget:              $225000000
		Meets Budget:         True
		Unallocated Money:   $188180454.54545453
	
	Constellation #1 Volume Summary:
		Payload Fits:     True
		Available Volume: 1.5 m^3
		Used Volume:      0.216 m^3
		Remaining Volume: 1.284 m^3
	
	Constellation #1 Visitation Summary:
		Average visitations per day: 8.0
	
	Constellation #1 Point Coverage:
		Can Point Within SW:     True
		% Coverage:              0.04225%
		Points Sampled:          4000
		Points in range and LOS: 169
		Points in range:         169
		Points in LOS:           675
		Closest Approach:        598.0135220827832 km
"""


# load in our orbits from the corrected MSII 
old_orbit1 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 290.375,
	w    = 90,
	v    = 0.0
)
old_orbit2 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 300.375,
	w    = 90,
	v    = 126.19563368939261
)
old_orbit3 = MSIII_Orbit( 
	a    = 8058.997698797144,
	e    = 0.13399441706731352,
	i    = 53.25,
	raan = 310.375,
	w    = 90,
	v    = 132.1759947994688
)

old_sat1 = MSIII_Satellite() # create new satellites for our constellation
old_sat2 = MSIII_Satellite()
old_sat3 = MSIII_Satellite()
old_sat1.orbit = old_orbit1 # force the MSII corrected orbits into the new satellites
old_sat2.orbit = old_orbit2
old_sat3.orbit = old_orbit3



const_II = MSIII_Constellation({
	"payload": PAYLOADS[0],          # Payload TinyCam (cheapest)
	"structure": STRUCTURES[0],      # Structure Option I
	"adcs": ADCS[1],                 # ADCS Option II
	"sats": []                       # no satellites, we will add these in next
})
const_II.sats = [old_sat1, old_sat2, old_sat3] # force the correct satellites into the constellation

Using default satellite constructor, make sure to add orbits.
Using default satellite constructor, make sure to add orbits.
Using default satellite constructor, make sure to add orbits.


In [18]:
# now we need to nerf the orbits (put them all on the same orbit, except for true anomaly)

def nerf_constellation(c : MSIII_Constellation):
    orbit1 = c.sats[0].orbit # pick the first orbit as the dominant orbit
    orbit1 : MSIII_Orbit
    for satellite in c.sats: # force all the other orbit to the dominant orbit's COEs
        orbit = satellite.orbit
        orbit : MSIII_Orbit
        orbit.a = orbit1.a
        orbit.e = orbit1.e
        orbit.raan = orbit1.raan
        orbit.i = orbit1.i
        orbit.w = orbit1.w

const3 = const_II
const3 : MSIII_Constellation
nerf_constellation(const_II) # nerf it


# assess the damage
ner_name = "(Nerfed)"
print("="*10 + "Post-nerf damage assessment" + "="*10)
print("\n")
print(const3.to_string(ner_name))
print(const3.get_concise_summary().to_string(ner_name))
print(const3.assess_visitation().to_string(ner_name))

==========Post-nerf damage assessment==========


Constellation (Nerfed):
	Payload:   Tiny Cam
	Structure: Option I
	ADCS:      Option II
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 126.19563368939261
	ORBIT: 
		a    : 8058.997698797144
		e    : 0.13399441706731352
		i    : 53.25
		raan : 290.375
		w    : 90
		v    : 132.1759947994688

Constellation (Nerfed) Concise Summary:
	Passes:       True
	In Budget:    True
	Viable Orbit: True
	Volume Fits:  True

Constellation (Nerfed) Visitation Summary:
	Average visitations per day: 8.5



This is strange, our visitation numbers actually went up. Upon further review of the visitation assessment implementation, it is still completely correct. Perhaps this was not the most reliable metric, but we will be sticking with it at this point to reduce design complexity. Possible reasons why this visitation number might have gone up include the potential for a point to briefly exit and then re-enter the swath width at perigee, thus counting two visitations. This is unclear, but will not be investigated further. A better metric to avoid this issue may have been to calculate the average squared duration between visits (squared so that longer waits would be penalized more heavily) and minimize that number. 

All this discussion aside, we now have our orbits.

### 